In [228]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report,mean_absolute_error,r2_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [25]:
RNST = 1373
DF = pd.read_csv('../../datasets/Automobile.csv',low_memory=False,
                 usecols=[
                     'bore','fuel_type','number_of_doors','body_style','engine_location','engine_type','engine_size',
                     'fuel_system','bore','horsepower','peak_rpm','city_mpg','highway_mpg',
                     'compression_ratio','stroke','price'
                 ])

In [ ]:
# lbe_fuel_type = LabelEncoder()
# DF['fuel_type'] = lbe_fuel_type.fit_transform(DF['fuel_type'])

# lbe_number_of_doors = LabelEncoder()
# DF['number_of_doors'] = lbe_number_of_doors.fit_transform(DF['number_of_doors'])

# lbe_body_style = LabelEncoder()
# DF['body_style'] = lbe_body_style.fit_transform(DF['body_style'])

# lbe_engine_location = LabelEncoder()
# DF['engine_location'] = lbe_engine_location.fit_transform(DF['engine_location'])

# lbe_engine_type = LabelEncoder()
# DF['engine_type'] = lbe_engine_type.fit_transform(DF['engine_type'])

# lbe_fuel_system = LabelEncoder()
# DF['fuel_system'] = lbe_fuel_system.fit_transform(DF['fuel_system'])


In [4]:
def OHC_transformer_single_column(DF,column_name,merged = False):
    label_encoder = LabelEncoder()
    encoded = label_encoder.fit_transform(DF[column_name])
    labels =pd.Series(label_encoder.classes_)
    
    clmt = ColumnTransformer([(column_name,OneHotEncoder(),[0])], remainder='passthrough')
    clmt_fited_data = clmt.fit_transform(DF[[column_name]]) 
    clmt_fited_data = clmt_fited_data if type(clmt_fited_data)== np.ndarray else clmt_fited_data.toarray()  
    NDF = pd.DataFrame(clmt_fited_data,columns=[f'{column_name}_{x}' for x in labels])
    
    if merged is True :
        merged = DF.merge(NDF, left_index=True,right_index=True)
        return merged.drop(columns=[column_name])
    else: 
        return NDF

In [27]:
PPDF = OHC_transformer_single_column(DF,'fuel_type',merged=True)
PPDF = OHC_transformer_single_column(PPDF,'number_of_doors',merged=True)
PPDF = OHC_transformer_single_column(PPDF,'body_style',merged=True)
PPDF = OHC_transformer_single_column(PPDF,'engine_location',merged=True)
PPDF = OHC_transformer_single_column(PPDF,'engine_type',merged=True)
PPDF = OHC_transformer_single_column(PPDF,'fuel_system',merged=True)

In [44]:
PPDF.columns

Index(['engine_size', 'bore', 'stroke', 'compression_ratio', 'horsepower',
       'peak_rpm', 'city_mpg', 'highway_mpg', 'price', 'fuel_type_diesel',
       'fuel_type_gas', 'number_of_doors_four', 'number_of_doors_two',
       'body_style_convertible', 'body_style_hardtop', 'body_style_hatchback',
       'body_style_sedan', 'body_style_wagon', 'engine_location_front',
       'engine_location_rear', 'engine_type_dohc', 'engine_type_l',
       'engine_type_ohc', 'engine_type_ohcf', 'engine_type_ohcv',
       'engine_type_rotor', 'fuel_system_1bbl', 'fuel_system_2bbl',
       'fuel_system_4bbl', 'fuel_system_idi', 'fuel_system_mfi',
       'fuel_system_mpfi', 'fuel_system_spdi', 'fuel_system_spfi'],
      dtype='object')

In [45]:
PPDF

,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price,fuel_type_diesel,...,engine_type_ohcv,engine_type_rotor,fuel_system_1bbl,fuel_system_2bbl,fuel_system_4bbl,fuel_system_idi,fuel_system_mfi,fuel_system_mpfi,fuel_system_spdi,fuel_system_spfi
0,130,3.47,2.68,9.0,111,5000,21,27,13495,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,130,3.47,2.68,9.0,111,5000,21,27,16500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,152,2.68,3.47,9.0,154,5000,19,26,16500,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,109,3.19,3.40,10.0,102,5500,24,30,13950,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,136,3.19,3.40,8.0,115,5500,18,22,17450,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,141,3.78,3.15,9.5,114,5400,23,28,16845,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
197,141,3.78,3.15,8.7,160,5300,19,25,19045,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
198,173,3.58,2.87,8.8,134,5500,18,23,21485,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
199,145,3.01,3.40,23.0,106,4800,26,27,22470,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [49]:
nnc = ['engine_size','bore','stroke','compression_ratio','horsepower','peak_rpm','city_mpg']

In [60]:
normalizer = Normalizer()
PPDF[nnc] = normalizer.fit_transform(PPDF[nnc])

In [61]:
X = PPDF[PPDF.columns.difference(['price'])]
Y = PPDF['price']
X_tain,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=RNST)

In [66]:
linreg = LinearRegression()
linreg.fit(X_tain,Y_train)
Y_predict_linreg = linreg.predict(X_test)
linreg.score(X_test,Y_test)

0.9146347763914926

In [226]:
r2_score(Y_test,Y_predict_linreg) # coefficient of determination 0 ~ 1 ==> 1 is better

0.9146347763914926

In [227]:
mean_absolute_error(Y_test,Y_predict_linreg)

1747.6131604232828

In [68]:
SVRreg = SVR(kernel='linear')
SVRreg.fit(X_tain,Y_train)
Y_predict_SVRreg = SVRreg.predict(X_test)
SVRreg.score(X_test,Y_test)

0.22172744917839549

In [69]:
knnreg = KNeighborsRegressor(n_neighbors=3,weights='distance',p=1)
knnreg.fit(X_tain,Y_train)
Y_predict_knnreg = knnreg.predict(X_test)
knnreg.score(X_test,Y_test)

0.7965060469047032

In [134]:
rnfreg = RandomForestRegressor(min_samples_leaf=6,max_features=19,ccp_alpha=100,random_state=RNST)
rnfreg.fit(X_tain,Y_train)
Y_predict_rnfreg = rnfreg.predict(X_test)
rnfreg.score(X_test,Y_test)

0.8705752500962165

In [188]:
adabreg = AdaBoostRegressor(random_state=RNST,learning_rate=.03,loss='exponential')
adabreg.fit(X_tain,Y_train)
Y_predict_adabreg = adabreg.predict(X_test)
adabreg.score(X_test,Y_test)

0.8410097303115919

In [225]:
d3reg = DecisionTreeRegressor(random_state=RNST,splitter='random',min_samples_split=20)
d3reg.fit(X_tain,Y_train)
Y_predict_d3reg = d3reg.predict(X_test)
d3reg.score(X_test,Y_test)

0.8419104913245345